### Semi Sulamericana


In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json
import re 

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-07-14 17:28:27,212 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


### Função de conversão de JS para dicionário Python

In [ ]:
def carregar_js_para_dict(caminho_arquivo):
    with open(caminho_arquivo, encoding='utf-8') as f:
        conteudo = f.read()
        # remove o "const nome = ...;"
        match = re.search(r'=\s*(\[\s*{.*}\s*])\s*;', conteudo, re.DOTALL)
        if match:
            return json.loads(match.group(1))
        else:
            raise ValueError("Formato do arquivo inválido")

### Função para gerar confrontos da próxima fase

In [3]:
def gerar_confrontos_proxima_fase(confrontos_atual, resultados_atual, rodada_ida, rodada_volta, rodada_proxima_fase):
    jogos = sorted(list(set([c['jogo'] for c in confrontos_atual])))
    classificados = []

    for jogo in jogos:
        ida = next((r for r in resultados_atual if r['jogo'] == jogo and r['rodada'] == rodada_ida), None)
        volta = next((r for r in resultados_atual if r['jogo'] == jogo and r['rodada'] == rodada_volta), None)

        if not ida or not volta:
            continue

        time1 = ida['mandante']['nome']
        time2 = ida['visitante']['nome']

        pontos1 = ida['mandante']['pontos'] + volta['visitante']['pontos']
        pontos2 = ida['visitante']['pontos'] + volta['mandante']['pontos']

        vencedor = time1 if pontos1 > pontos2 else time2
        classificados.append(vencedor)

    confrontos_novos = []
    for i in range(0, len(classificados), 2):
        confronto = {
            "jogo": f"Jogo {chr(65 + i // 2)}",
            "rodada": rodada_proxima_fase,
            "mandante": {"nome": classificados[i]},
            "visitante": {"nome": classificados[i + 1]}
        }
        confrontos_novos.append(confronto)

    return confrontos_novos


### Carregar os dados das quartas

In [ ]:
confrontos_quartas = carregar_js_para_dict("confrontos_quartas_sula.js")
resultados_quartas = carregar_js_para_dict("resultados_quartas_sula.js")

### Gerar confrontos da semi

In [9]:
confrontos_semi = gerar_confrontos_proxima_fase(
    confrontos_quartas,
    resultados_quartas,
    rodada_ida=11,
    rodada_volta=12,
    rodada_proxima_fase=13
)

### Salvar confrontos_semi_sula.js

In [16]:
# Salvar o arquivo JS normalmente
with open("confrontos_semi_sula.js", "w", encoding="utf-8") as f:
    f.write("const confrontos_semi_sula = ")
    json.dump(confrontos_semi, f, indent=2, ensure_ascii=False)
    f.write(";")

# Transformar em DataFrame para exportar para Excel
df_confrontos = pd.DataFrame([
    {
        "jogo": c["jogo"],
        "rodada": c["rodada"],
        "mandante": c["mandante"]["nome"],
        "visitante": c["visitante"]["nome"]
    }
    for c in confrontos_semi
])

# Salvar em Excel
df_confrontos.to_excel("confrontos_semi_sula.xlsx", index=False)

# Visualização
display(df_confrontos)
print("Confrontos da semifinal salvos em .js e .xlsx com sucesso!")


,jogo,rodada,mandante,visitante
0,Jogo A,13,Gremiomaniasm,Grêmio imortal 37
1,Jogo B,13,MauHumor F.C.,E.C. Bororé


Confrontos da semifinal salvos em .js e .xlsx com sucesso!


In [14]:
import json

with open("confrontos_semi_sula.js", encoding="utf-8") as f:
    conteudo = f.read()
    confrontos_semi = json.loads(conteudo.split("=", 1)[1].strip(" ;"))


In [17]:
def exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada, jogo=None):
    """
    Exibe os resultados de uma rodada específica, com pontuação e dados dos times.
    """

    if "Rodada" not in df_rodadas.columns:
        raise ValueError("O DataFrame df_rodadas precisa conter a coluna 'Rodada'.")

    if rodada not in df_rodadas["Rodada"].values:
        return pd.DataFrame([{
            "Jogo": jogo or "-",
            "Rodada": rodada,
            "Mandante_Nome": "-",
            "Mandante_Clube": "-",
            "Mandante_Participante": "-",
            "Mandante_Pontos": "-",
            "Visitante_Nome": "-",
            "Visitante_Clube": "-",
            "Visitante_Participante": "-",
            "Visitante_Pontos": "-",
        }])

    df_filtrado = df_rodadas[df_rodadas["Rodada"] == rodada]
    if jogo:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    resultados = []

    for _, row in df_filtrado.iterrows():
        jogo_ = row["Jogo"]
        mandante = row["Mandante_Nome"]
        visitante = row["Visitante_Nome"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "Jogo": jogo_,
            "Rodada": rodada,
            "Mandante_Nome": mandante,
            "Mandante_Pontos": pontos_mandante,
            "Visitante_Nome": visitante,
            "Visitante_Pontos": pontos_visitante
        })

    return pd.DataFrame(resultados)



In [ ]:
# Exibir resultados da 2ª rodada
df_resultados_rodada_1 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=11)

# Exibir apenas os resultados do Grupo B na 1ª rodada
df_resultados_jogo_B = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=11, jogo="Jogo B1")

# Exibir
display(df_resultados_rodada_1)
display(df_resultados_jogo_B)